In [18]:
%%writefile assignment44.cpp
// Assignment 4 Task 4
// Реализовать распределённую программу с использованием MPI
// Разделить массив между процессами, выполнить вычисления локально и собрать результаты
// Замеры времени выполнения для 2, 4 и 8 процессов.
#include <iostream>                               // Для стандартный ввод-вывод
#include <mpi.h>                                  // MPI библиотека для распределённых вычислений
#include <vector>                                 // Для динамического массива
using namespace std;                              // Чтобы не писать std::

int main(int argc, char *argv[]) {
    MPI_Init(&argc, &argv);                        // Инициализация MPI среды

    int world_rank;                                // Ранг текущего процесса
    int world_size;                                // Общее количество процессов

    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);    // Получаем ранг текущего процесса
    MPI_Comm_size(MPI_COMM_WORLD, &world_size);    // Получаем общее количество процессов

    const int N = 1000000;                         // Размер массива
    vector<int> array;                             // Массив для хранения данных

    if (world_rank == 0) {                         // Только процесс с рангом 0 инициализирует массив
        array.resize(N);                           // Выделяем память под массив
        for (int i = 0; i < N; i++)                // Заполняем массив единицами
            array[i] = 1;
    }

    int local_size = N / world_size;               // Размер части массива для каждого процесса
    vector<int> local_array(local_size);           // Локальный массив для каждого процесса

    // Разделяем массив между процессами
    MPI_Scatter(array.data(), local_size, MPI_INT, // Отправляем по частям
                local_array.data(), local_size, MPI_INT,                        // Буфер приёма на каждом процессе (куда MPI положит свою часть)
                                                                                // Количество элементов, которые каждый процесс примет
                                                                                // Тип данных приёма (должен совпадать с отправкой)
                0, MPI_COMM_WORLD);                                             // Ранг процесса-отправителя (root) — здесь процесс 0
                                                                                // Коммуникатор, в рамках которого выполняется scatter

    // Измеряем время локальных вычислений
    double start_time = MPI_Wtime();               // Старт таймера MPI

    int local_sum = 0;                             // Локальная сумма на каждом процессе
    for (int i = 0; i < local_size; i++)          // Проходим по локальной части
        local_sum += local_array[i];              // Суммируем элементы

    int global_sum = 0;                            // Переменная для хранения глобальной суммы

    // Собираем результаты с всех процессов на процесс 0
    MPI_Reduce(&local_sum, &global_sum, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);

    double end_time = MPI_Wtime();                 // Остановка таймера

    // Процесс 0 выводит результат
    if (world_rank == 0) {
        cout << "Глобальная сумма: " << global_sum << endl;
        cout << "Время выполнения с " << world_size << " процессами: "
             << (end_time - start_time) * 1000 << " мс" << endl;
    }

    MPI_Finalize();                                // Завершаем MPI
    return 0;                                      // Завершаем
}


Overwriting assignment44.cpp


In [21]:
# Компиляция
!mpic++ assignment44.cpp -o assignment44
# mpirun — утилита, которая запускает MPI-программу на указанном числе процессов
# --allow-run-as-root — разрешает запуск MPI от root (Colab запускает ядра как root)
# --oversubscribe — игнорирует количество доступных виртуальных CPU, позволяя запускать больше процессов, чем физически есть
# -np 2 — число процессов MPI (2 процесса)

# Запуск с 2 процессами
!mpirun --allow-run-as-root --oversubscribe -np 2 ./assignment44

# Запуск с 4 процессами
!mpirun --allow-run-as-root --oversubscribe -np 4 ./assignment44

# Запуск с 8 процессами
!mpirun --allow-run-as-root --oversubscribe -np 8 ./assignment44


Глобальная сумма: 1000000
Время выполнения с 2 процессами: 2.45628 мс
Глобальная сумма: 1000000
Время выполнения с 4 процессами: 1.41423 мс
Глобальная сумма: 1000000
Время выполнения с 8 процессами: 0.6472 мс
